In [55]:
import pandas as pd
import os
files=os.listdir('最终数据')
print(files)

['000001.csv', '000002.csv', '000004.csv', '000005.csv', '000006.csv', '000007.csv', '000008.csv', '000009.csv', '000010.csv', '000011.csv', '000012.csv', '000014.csv', '000016.csv', '000017.csv', '000019.csv', '000020.csv', '000021.csv', '000023.csv', '000025.csv', '000026.csv', '000027.csv', '000028.csv', '000030.csv', '000032.csv', '000034.csv', '000035.csv', '000036.csv', '000037.csv', '000038.csv', '000040.csv', '000042.csv', '000045.csv', '000046.csv', '000048.csv', '000049.csv', '000050.csv', '000055.csv', '000056.csv', '000058.csv', '000059.csv', '000061.csv', '000063.csv', '000065.csv', '000068.csv', '000078.csv', '000088.csv', '000090.csv', '000096.csv', '000099.csv', '000100.csv', '000150.csv', '000151.csv', '000155.csv', '000156.csv', '000157.csv', '000158.csv', '000159.csv', '000166.csv', '000301.csv', '000401.csv', '000402.csv', '000403.csv', '000404.csv', '000407.csv', '000408.csv', '000409.csv', '000410.csv', '000411.csv', '000413.csv', '000415.csv', '000419.csv', '0004

In [56]:
# result=pd.read_csv("最终数据/"+str(files[0]))
# for i in range(1,len(files)):
#     temp=pd.read_csv("最终数据/"+str(files[i]))
#     result=pd.concat([result,temp])

In [57]:
cols_not_null = result.count(axis=0)
cols_null = result.shape[0] - cols_not_null
print(cols_null/result.shape[0])

inv_turn                      0.030224
assets_turn                   0.000495
grossprofit_margin            0.016768
roe                           0.007733
roa                           0.014796
ocf_to_or                     0.014300
longdebt_to_workingcapital    0.263055
q_sales_yoy                   0.055495
q_op_yoy                      0.055587
equity_yoy                    0.036948
revenue                       0.905732
yoy_sales                     1.000000
yoy_op                        1.000000
growth_assets                 1.000000
trade_date                    0.022868
turnover_rate                 0.022868
dv_ratio                      0.376989
bp                            0.030004
ep                            0.087893
sp                            0.023280
return                        0.000000
jidu                          0.000000
code                          0.000000
dtype: float64


In [80]:
for i in range(0,len(files)):
    temp=pd.read_csv("最终数据/"+str(files[i]))
    temp=temp.drop(['revenue', 'yoy_sales','yoy_op','growth_assets','dv_ratio'], axis=1)
    cols_not_null = temp.count(axis=0)
    cols_null = (temp.shape[0] - cols_not_null)/temp.shape[0]
    check=list(cols_null)
    bole=all(i <= 0.5 for i in check)
    if bole:
        temp.to_csv('回归/'+str(files[i]),index=False)

In [81]:
files=os.listdir('回归')
from  sklearn import ensemble
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
def set_missing_ages(df):
	#把数值型特征都放到随机森林里面去
    fill_list=['q_sales_yoy','assets_turn','grossprofit_margin','roe','roa',
               'ocf_to_or','longdebt_to_workingcapital','inv_turn','q_op_yoy',
               'equity_yoy','trade_date','turnover_rate','bp','ep','sp']
    age_df=df[['q_sales_yoy','assets_turn','grossprofit_margin','roe','roa',
               'ocf_to_or','longdebt_to_workingcapital','inv_turn','q_op_yoy',
               'equity_yoy','trade_date','turnover_rate','bp',
               'ep','sp','return']]
    for feature in fill_list:
        if age_df[feature].count()==age_df.shape[0]:
            continue
        temp = age_df[feature]
        age_df = age_df.drop(feature,axis=1)
        age_df.insert(0,feature,temp)
        known_age = age_df[age_df[feature].notnull()].fillna(value=0)
        unknown_age = age_df[age_df[feature].isnull()].fillna(value=0)
        y=known_age.iloc[:,0]#y是第一列数据
        x=known_age.iloc[:,1:]#x是特征属性值，后面几列
        rfr=tree.DecisionTreeRegressor()
        #根据已有数据去拟合随机森林模型
        rfr.fit(x,y)
    #预测缺失值
        predictedAges = rfr.predict(unknown_age.iloc[:,1:])
    #填补缺失值
        df.loc[(df[feature].isnull()),feature] = predictedAges

    return df

In [82]:
for i in range(0,len(files)):
    temp=pd.read_csv("回归/"+str(files[i]))
    temp=set_missing_ages(temp)
    temp.to_csv('填补完成/'+str(files[i]),index=False)